In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from collections import deque, namedtuple
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import os
import gc

ImportError: DLL load failed while importing _C: The specified module could not be found.

In [ ]:
from pandas.api.types import CategoricalDtype

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define experience tuple structure
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """Experience replay buffer to store and sample agent experiences"""

    def __init__(self, buffer_size, batch_size):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory"""
        experience = Experience(state, action, reward, next_state, done)
        self.memory.append(experience)

    def sample(self):
        """Randomly sample a batch of experiences"""
        experiences = random.sample(self.memory, k=min(self.batch_size, len(self.memory)))

        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float().to(device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.memory)

class QTable:
    """Tabular Q-learning implementation for real estate decision making"""

    def __init__(self, state_bins, action_size, learning_rate=0.1, discount_factor=0.99):
        """Initialize Q-table parameters

        Args:
            state_bins (dict): Dictionary of feature names to number of bins
            action_size (int): Number of possible actions
            learning_rate (float): Learning rate alpha
            discount_factor (float): Discount factor gamma
        """
        self.state_bins = state_bins
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor

        # Create Q-table with dimensions based on state bins
        dimensions = list(state_bins.values()) + [action_size]
        self.q_table = np.zeros(dimensions)

        # Track feature names for discretization
        self.feature_names = list(state_bins.keys())

    def discretize_state(self, state):
        """Convert continuous state to discrete state for Q-table"""
        if isinstance(state, np.ndarray) and state.dtype == np.dtype('O'):
            state = np.vstack(state).astype(np.float32)

        # Ensure state is 1-dimensional
        if len(state.shape) > 1:
            state = state.flatten()

        # Use fewer dimensions to make Q-table manageable
        # Select most important features (first 5 features)
        if len(state) > 5:
            state = state[:5]

        # Discretize each dimension
        discrete_state = []
        for i, val in enumerate(state):
            # Check for NaN values before conversion
            if np.isnan(val):
                bin_idx = 0  # Default bin for NaN values
            else:
                bin_idx = min(max(0, int(val * self.state_bins)), self.state_bins - 1)
            discrete_state.append(bin_idx)

        return tuple(discrete_state)



    def get_value(self, state, action=None):
        """Get Q-value for state and action, or all action values if action=None"""
        if action is None:
            return self.q_table[state]
        return self.q_table[state + (action,)]

    def update(self, state, action, reward, next_state):
        """Update Q-value using Q-learning update rule"""
        current_q = self.q_table[state + (action,)]
        max_next_q = np.max(self.q_table[next_state])

        # Q-learning update formula: Q(s,a) = Q(s,a) + α[r + γ*max_a'Q(s',a') - Q(s,a)]
        new_q = current_q + self.learning_rate * (
            reward + self.discount_factor * max_next_q - current_q)

        self.q_table[state + (action,)] = new_q

class RealEstateAgent:
    """Agent that interacts with and learns from the real estate environment using Q-Learning"""

    def __init__(self, state_size, action_size, seed=42):
        """Initialize a Q-Learning Agent object"""
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-table discretization parameters
        self.state_bins = 10  # Number of bins per state dimension

        # Create Q-table (discretized state space)
        # Using a dictionary for sparse representation
        self.q_table = {}

        # Hyperparameters
        self.alpha = 0.1          # Learning rate
        self.gamma = 0.99         # Discount factor
        self.epsilon = 1.0        # Exploration rate
        self.epsilon_decay = 0.995  # Decay rate
        self.epsilon_min = 0.01   # Minimum exploration rate

        # Portfolio tracking (unchanged)
        self.portfolio = {}
        self.cash = 1000000
        self.net_worth_history = []
        self.net_worth_threshold_1 = 2000000
        self.net_worth_threshold_2 = 5000000

    def discretize_state(self, state):
        """Convert continuous state to discrete state for Q-table"""
        if isinstance(state, np.ndarray) and state.dtype == np.dtype('O'):
            state = np.vstack(state).astype(np.float32)

        # Ensure state is 1-dimensional
        if len(state.shape) > 1:
            state = state.flatten()

        # Use fewer dimensions to make Q-table manageable
        # Select most important features (first 5 features)
        if len(state) > 5:
            state = state[:5]

        # Discretize each dimension
        discrete_state = []
        for i, val in enumerate(state):
            # Replace NaN with 0 using NumPy's nan_to_num
            val = np.nan_to_num(val)
            bin_idx = min(max(0, int(val * self.state_bins)), self.state_bins - 1)
            discrete_state.append(bin_idx)

        return tuple(discrete_state)

    def step(self, state, action, reward, next_state, done):
        """Update Q-values using Q-Learning update rule"""
        # Discretize states for Q-table lookup
        discrete_state = self.discretize_state(state)
        discrete_next_state = self.discretize_state(next_state)

        # Initialize Q-values if states not seen before
        if discrete_state not in self.q_table:
            self.q_table[discrete_state] = np.zeros(self.action_size)

        if discrete_next_state not in self.q_table:
            self.q_table[discrete_next_state] = np.zeros(self.action_size)

        # Q-Learning update formula: Q(s,a) ← Q(s,a) + α[r + γ·max(Q(s',a')) - Q(s,a)]
        best_next_action = np.argmax(self.q_table[discrete_next_state])
        td_target = reward + self.gamma * self.q_table[discrete_next_state][best_next_action]
        td_error = td_target - self.q_table[discrete_state][action]
        self.q_table[discrete_state][action] += self.alpha * td_error

        # Decay exploration rate
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def act(self, state, eval_mode=False):
        """Select action based on current policy (epsilon-greedy)"""
        # Discretize state for Q-table lookup
        discrete_state = self.discretize_state(state)

        # Set exploration probability
        if eval_mode:
            epsilon = 0.0  # No exploration during evaluation
        else:
            epsilon = self.epsilon

        # Epsilon-greedy action selection
        if random.random() > epsilon:
            # Exploitation: choose best action from Q-table
            if discrete_state in self.q_table:
                return np.argmax(self.q_table[discrete_state])
            else:
                # If state not in Q-table, initialize and choose randomly
                self.q_table[discrete_state] = np.zeros(self.action_size)
                return random.choice(np.arange(self.action_size))
        else:
            # Exploration: choose random action
            return random.choice(np.arange(self.action_size))

    def update_portfolio(self, prop_id, action, price, year):
        """
        Update the agent's portfolio based on the action taken

        Args:
            prop_id: The property identifier
            action: 'buy' or 'sell'
            price: The transaction price
            year: The current year in the simulation

        Returns:
            If selling: returns the profit/loss amount
            If buying: returns True if successful, False otherwise
        """
        if action == 'buy':
            # Check if we have enough cash
            if self.cash < price:
                return False

            # Add property to portfolio with recommended holding period
            self.portfolio[prop_id] = {
                'purchase_price': price,
                'purchase_year': year,
                'recommended_holding_period': random.randint(3, 8)  # Random holding period
            }

            # Deduct price from cash
            self.cash -= price
            return True

        elif action == 'sell':
            # Check if we own the property
            if prop_id not in self.portfolio:
                return 0  # No profit if we don't own the property

            # Calculate profit
            purchase_price = self.portfolio[prop_id]['purchase_price']
            profit = price - purchase_price

            # Add sale price to cash
            self.cash += price

            # Remove property from portfolio
            del self.portfolio[prop_id]
            return profit

        return 0  # Default return for invalid actions


    def record_net_worth(self, property_values):
        """
        Calculate and record the agent's net worth based on cash and property values

        Args:
            property_values (dict): Dictionary mapping property IDs to their current values

        Returns:
            float: The agent's current net worth
        """
        # Calculate total property value
        total_property_value = sum(property_values.values())

        # Calculate net worth (cash + property value)
        net_worth = self.cash + total_property_value

        # Record in history
        self.net_worth_history.append(net_worth)

        return net_worth

    def display_portfolio_details(self, data_processor):
        """
        Display detailed information about the agent's portfolio

        Args:
            data_processor: The data processor object to get property information
        """
        if not self.portfolio:
            print("Portfolio is empty.")
            return

        print("\nPortfolio Details:")
        print(f"Total Properties: {len(self.portfolio)}")
        print(f"Cash: ${self.cash:,.2f}")

        total_invested = 0
        for prop_id, details in self.portfolio.items():
            total_invested += details['purchase_price']

        print(f"Total Invested: ${total_invested:,.2f}")

        if self.net_worth_history:
            print(f"Current Net Worth: ${self.net_worth_history[-1]:,.2f}")
            initial_worth = self.net_worth_history[0] if len(self.net_worth_history) > 1 else 1000000
            profit = self.net_worth_history[-1] - initial_worth
            print(f"Profit: ${profit:,.2f} ({profit/initial_worth:.2%})")

        print("\nProperty Details:")
        for prop_id, details in self.portfolio.items():
            purchase_price = details['purchase_price']
            purchase_year = details['purchase_year']
            holding_period = details.get('recommended_holding_period', 5)
            print(f"Property {prop_id}: Purchased for ${purchase_price:,.2f} in year {purchase_year}, Recommended holding: {holding_period} years")


class RealEstateDataProcessor:
    """Class to handle real estate data loading, processing and feature engineering"""

    def __init__(self, data_dir='./'):
        """Initialize the data processor

        Args:
            data_dir (str): Directory containing the dataset files
        """
        self.data_dir = data_dir
        self.scaler = None
        self.datasets = {}
        self.property_sets = {}  # For train/val/test/prod splits
        self.last_updated_years = {}  # Track when each property was last updated
        self.property_values_by_year = {}  # Track property values by year

    def load_datasets(self):
        """Load all dataset files"""
        print("Loading datasets...")

        try:
            self.datasets = {}

            self.datasets['mortgage_info'] = reduce_memory_usage(pd.read_csv(os.path.join(self.data_dir, 'listing_mortgage_info.csv')))
            self.datasets['nearby_homes'] = reduce_memory_usage(pd.read_csv(os.path.join(self.data_dir, 'listing_nearby_homes.csv')))
            self.datasets['price_history'] = reduce_memory_usage(pd.read_csv(os.path.join(self.data_dir, 'listing_price_history.csv')))
            self.datasets['schools_info'] = reduce_memory_usage(pd.read_csv(os.path.join(self.data_dir, 'listing_schools_info.csv')))
            self.datasets['subtype'] = reduce_memory_usage(pd.read_csv(os.path.join(self.data_dir, 'listing_subtype.csv')))
            self.datasets['tax_info'] = reduce_memory_usage(pd.read_csv(os.path.join(self.data_dir, 'listing_tax_info.csv')))
            self.datasets['property_listings'] = reduce_memory_usage(pd.read_csv(os.path.join(self.data_dir, 'property_listings.csv')))

            print("All datasets loaded successfully!")
            print(f"Main property dataset shape: {self.datasets['property_listings'].shape}")

            # Extract historical years from price_history
            self.datasets['price_history']['dateOfEvent'] = pd.to_datetime(self.datasets['price_history']['dateOfEvent'])
            price_history_years = self.datasets['price_history'][
                (self.datasets['price_history']['dateOfEvent'].dt.year >= 2010) &
                (self.datasets['price_history']['dateOfEvent'].dt.year <= 2024)
            ]
            
            # Group by property and count unique years
            prop_year_counts = price_history_years.groupby('zpid')['dateOfEvent'] \
                .apply(lambda x: x.dt.year.nunique())
            
            min_required_years = 7
            
            # Filter properties with at least min_required_years years of data 
            valid_properties = prop_year_counts[prop_year_counts >= min_required_years].index.tolist()

            # If still no valid properties, use any with at least 1 year
            if len(valid_properties) == 0:
                valid_properties = price_history_years['zpid'].unique()
                print(f"Using {len(valid_properties)} properties with any historical data")
            
            # Then proceed with intersection check
            valid_properties = np.intersect1d(
                valid_properties,
                self.datasets['property_listings']['zpid'].unique()
            )
            
            # Additional fallback if still no properties
            if valid_properties.size == 0:
                print("No properties with any price history found - using all properties")
                valid_properties = self.datasets['property_listings']['zpid'].unique()

            
            # Intersection with property listings
            valid_properties = np.intersect1d(
                valid_properties,
                self.datasets['property_listings']['zpid'].unique()
            )
            
            valid_properties = np.array(valid_properties)
            
            if valid_properties.size == 0:
                # Fallback: Use properties with ANY 2010-2024 history if none have full 15 years
                valid_properties = price_history_years['zpid'].unique()
                print(f"Warning: No properties with full 2010-2024 history. Using {len(valid_properties)} with partial data")

            # Then filter all datasets to only include these properties
            for dataset_name in self.datasets:
                if isinstance(self.datasets[dataset_name], pd.DataFrame) and 'zpid' in self.datasets[dataset_name].columns:
                    self.datasets[dataset_name] = self.datasets[dataset_name][
                        self.datasets[dataset_name]['zpid'].isin(valid_properties)]


            print(f"Valid Properties: {len(valid_properties)}")
            # Filter for years 2000 and later
            valid_years = sorted(price_history_years['zpid'].unique())
            valid_years = [year for year in valid_years if year >= 2010 and year <= 2024]
            print(f"Filtered to {len(valid_years)} properties with price history in 2010")

            # Extract current property years from lastUpdated
            self.datasets['property_listings']['year'] = pd.to_datetime(
                self.datasets['property_listings']['lastUpdated']).dt.year

            # Create a mapping of zpid to all its available years from both datasets
            property_years = {}

            # First add years from price_history (historical data)
            for zpid, year in zip(self.datasets['price_history']['zpid'],
                                  self.datasets['price_history']['dateOfEvent'].dt.year):
                if year >= 2000:  # Only include years from 2000 onwards
                    if zpid not in property_years:
                        property_years[zpid] = set()
                    property_years[zpid].add(year)

            # Then add current years from property_listings
            for zpid, year in zip(self.datasets['property_listings']['zpid'],
                                  self.datasets['property_listings']['year']):
                if zpid in property_years:  # Only add if property exists in historical data
                    property_years[zpid].add(year)

            # Use all years from 2000 to present
            self.years = sorted([year for year in valid_years if year >= 2010])

            if self.years:
                print(f"Dataset spans years: {min(self.years)} to {max(self.years)}")
                print(f"Total years available: {len(self.years)}")
            else:
                print("No valid years found in the dataset.")


            return True

        except FileNotFoundError as e:
            print(f"Error loading datasets: {e}")
            print("Please ensure all required CSV files are in the specified directory.")
            return False


    def split_data_by_properties(self):
        """Split data by properties while maintaining the full timeline
        Uses property-based split instead of time-based split.
        Only considers 30% of properties for all processing.
        Training set only includes properties from 5 randomly chosen states.
        """
        if not self.datasets:
            print("Datasets not loaded. Call load_datasets() first.")
            return

        # Check if property_listings exists
        if 'property_listings' not in self.datasets:
            print("Error: 'property_listings' dataset not found. Make sure the CSV file exists and was loaded correctly.")
            return

        # Check if 'zpid' column exists in property_listings
        if 'zpid' not in self.datasets['property_listings'].columns:
            print("Error: 'zpid' column not found in 'property_listings'. Check the column names in your CSV file.")
            print(f"Available columns: {self.datasets['property_listings'].columns.tolist()}")
            return
        
        # Filter price history to Only 2010 entries
        valid_price_history = self.datasets['price_history'][
            self.datasets['price_history']['dateOfEvent'].dt.year == 2010
        ]
        
        # Get properties with both 2010 price history AND current listings
        valid_properties = np.intersect1d(
            valid_price_history['zpid'].unique(),
            self.datasets['property_listings']['zpid'].unique()
        )
        
        if not valid_properties.size:
            raise ValueError("No properties with 2010 price history found")
        
        # Get properties with price history 2010-2024 (renamed variable for clarity)
        # price_history = self.datasets['price_history'].copy()
        # price_history['dateOfEvent'] = pd.to_datetime(price_history['dateOfEvent'])
        
        # valid_price_history = price_history[
        #     (price_history['dateOfEvent'].dt.year >= 2010) &
        #     (price_history['dateOfEvent'].dt.year <= 2024)
        # ]
        # 
        # # Get intersection of properties with both listing info and price history
        # valid_properties = np.intersect1d(
        #     valid_price_history['zpid'].unique(),
        #     self.datasets['property_listings']['zpid'].unique()
        # )
        # 
        # # After getting valid_properties from np.intersect1d()
        # if not len(valid_properties):
        #     raise ValueError("No valid properties with price history 2010-2024 found")
        
        print(f"Valid properties: {len(valid_properties)}")
        # Rename variable for clarity
        all_properties = valid_properties  # Only after validation

        # # Check if 'yearBuilt' column exists
        # is_2010 = self.datasets['property_listings']['yearBuilt'] == 2010
        # if 'yearBuilt' not in self.datasets['property_listings'].columns:
        #     print("Warning: 'yearBuilt' column not found in 'property_listings'. Skipping year-based filtering.")
        #     all_properties = self.datasets['property_listings']['zpid'].unique()
        # else:
        #     # Get all unique property IDs
        #     all_properties = self.datasets['property_listings']['zpid'].unique()
        # 
        #     # In split_data_by_properties():
        #     if 'yearBuilt' in self.datasets['property_listings'].columns:
        #         valid_properties = self.datasets['property_listings'].loc[
        #             self.datasets['property_listings']['yearBuilt'] == 2010, 'zpid'
        #         ].unique()
        # 
        #         if len(valid_properties) == 0:
        #             print("No 2010 properties found, using all properties")
        #             valid_properties = self.datasets['property_listings']['zpid'].unique()
        #     else:
        #         valid_properties = self.datasets['property_listings']['zpid'].unique()

        # Shuffle the properties for random assignment
        np.random.shuffle(all_properties)

        # Select only 30% of properties for consideration
        considered_count = int(len(all_properties) * 0.3)
        considered_properties = all_properties[:considered_count]
        print(f"Considering only {considered_count} properties ({considered_count/len(all_properties):.1%} of total)")

        # Check if 'state' column exists
        if 'state' not in self.datasets['property_listings'].columns:
            print("Warning: 'state' column not found in 'property_listings'. Using random assignment instead.")
            # Assign random "states" for illustration
            property_to_state = {prop_id: f"State_{i%10}" for i, prop_id in enumerate(considered_properties)}
        else:
            # Create a mapping of property IDs to their states
            property_to_state = {}
            for prop_id, state in zip(self.datasets['property_listings']['zpid'],
                                    self.datasets['property_listings']['state']):
                if prop_id in considered_properties:
                    property_to_state[prop_id] = state

        # Get unique states from considered properties
        available_states = list(set(property_to_state.values()))
        print(f"Available states in considered properties: {len(available_states)}")

        # Randomly select 5 states for training (or fewer if there are less than 5 states)
        training_states = random.sample(available_states, min(5, len(available_states)))
        print(f"Selected states for training: {training_states}")

        # Calculate sizes for splits
        train_size = int(len(considered_properties) * 0.6)
        val_size = int(len(considered_properties) * 0.15)
        test_size = int(len(considered_properties) * 0.15)

        # Filter considered properties by state for training
        train_state_properties = [p for p in considered_properties if property_to_state.get(p) in training_states]

        # Shuffle the filtered properties
        np.random.shuffle(train_state_properties)

        # Select properties for training (up to the original train size)
        train_props = train_state_properties[:min(train_size, len(train_state_properties))]

        # Select remaining properties for val, test, prod from all considered properties
        remaining_props = [p for p in considered_properties if p not in train_props]
        np.random.shuffle(remaining_props)

        self.property_sets = {
            'train': np.array(train_props),
            'val': np.array(remaining_props[:min(val_size, len(remaining_props))]),
            'test': np.array(remaining_props[val_size:min(val_size + test_size, len(remaining_props))]),
            'prod': np.array(remaining_props[min(val_size + test_size, len(remaining_props)):])
        }

        # Print statistics
        for split, props in self.property_sets.items():
            print(f"{split} set: {len(props)} properties ({len(props)/len(all_properties) if len(all_properties) > 0 else 0:.1%} of total)")

        # Print states distribution for training
        train_states_used = [property_to_state.get(prop_id) for prop_id in self.property_sets['train']]
        train_states_count = {state: train_states_used.count(state) for state in set(train_states_used)}
        print(f"Training set states distribution: {train_states_count}")

        return self.property_sets




    def engineer_features(self):
        """Engineer features with focus on historical price data integration"""
        if not self.datasets or not self.property_sets:
            print("Datasets not loaded or not split. Call load_datasets() and split_data_by_properties() first.")
            return

        print("Engineering features with historical data integration...")

        # Get all properties from our splits (the 30% considered)
        considered_properties = np.concatenate(list(self.property_sets.values()))

        # 1. Start with price history to create a base DataFrame with historical data
        price_history = self.datasets['price_history'].copy()
        # Filter to only include our considered properties
        price_history = price_history[price_history['zpid'].isin(considered_properties)]

        price_history['dateOfEvent'] = pd.to_datetime(price_history['dateOfEvent'])
        price_history['year'] = price_history['dateOfEvent'].dt.year

        # Filter for years 2000 and later
        price_history = price_history[price_history['year'] >= 2000]

        # Group by zpid and year to get price per year (if multiple events per year)
        yearly_prices = price_history.groupby(['zpid', 'year'])['price'].mean().reset_index()

        # Calculate year-over-year price changes
        yearly_prices = yearly_prices.sort_values(['zpid', 'year'])
        yearly_prices['prev_price'] = yearly_prices.groupby('zpid')['price'].shift(1)
        yearly_prices['price_yoy_change'] = (yearly_prices['price'] - yearly_prices['prev_price']) / yearly_prices['prev_price']

        # 2. Get property information from property_listings
        properties = self.datasets['property_listings'].copy()
        # Filter to only include considered properties
        properties = properties[properties['zpid'].isin(considered_properties)]

        properties['lastUpdated'] = pd.to_datetime(properties['lastUpdated'])
        properties['year_updated'] = properties['lastUpdated'].dt.year

        # 3. Create a base DataFrame with all property-year combinations
        # Use only our considered properties
        all_properties = considered_properties

        # Get all years from 2000 to present
        year_updated_max = properties['year_updated'].max()
        price_history_max = price_history['year'].max()

        # Handle potential NaN values
        if pd.isna(year_updated_max):
            year_updated_max = 2000  # Default fallback value
        if pd.isna(price_history_max):
            price_history_max = 2000  # Default fallback value

        all_years = list(range(2000, int(max(year_updated_max, price_history_max)) + 1))

        # Create a cross-product of properties and years
        property_years = []
        for prop in all_properties:
            for year in all_years:
                property_years.append({'zpid': prop, 'year': year})

        # Create base DataFrame
        df = pd.DataFrame(property_years)

        # 4. Merge property information
        # First get the latest property info
        latest_properties = properties.sort_values('lastUpdated').groupby('zpid').last().reset_index()

        # Merge with base DataFrame
        print(df.columns)
        df = pd.merge(df, latest_properties[['zpid', 'price', 'livingArea', 'bedrooms', 'bathrooms',
                                          'yearBuilt', 'homeType', 'homeStatus', 'state',
                                          'pageViewCount', 'favoriteCount']],
                    on='zpid', how='left')

        # 5. Merge historical prices
        df = pd.merge(df, yearly_prices[['zpid', 'year', 'price_yoy_change']],
                    on=['zpid', 'year'], how='left')

        # For historical years, update property price using the price from price_history
        historical_prices = pd.merge(df[['zpid', 'year']],
                                    yearly_prices[['zpid', 'year', 'price']],
                                    on=['zpid', 'year'], how='left')

        # Update prices where historical data exists
        mask = historical_prices['price'].notna()
        if mask.any():
            df.loc[mask, 'price'] = historical_prices.loc[mask, 'price'].values

        # 6. Add basic features
        df.loc[:, 'price_per_sqft'] = df['price'] / df['livingArea']
        df.loc[:, 'age'] = df['year'] - df['yearBuilt']
        df.loc[:, 'bed_bath_ratio'] = df['bedrooms'] / df['bathrooms']

        # 7. Add tax and mortgage features
        tax_info = self.datasets['tax_info'][['zpid', 'taxPaid', 'valueIncreaseRate']].copy()
        # Filter to only include considered properties
        tax_info = tax_info[tax_info['zpid'].isin(considered_properties)]
        df = pd.merge(df, tax_info, on='zpid', how='left')

        mortgage_info = self.datasets['mortgage_info'][['zpid', 'rate']].copy()
        # Filter to only include considered properties
        mortgage_info = mortgage_info[mortgage_info['zpid'].isin(considered_properties)]
        df = pd.merge(df, mortgage_info, on='zpid', how='left')

        # 8. Add school ratings
        schools = self.datasets['schools_info'][['zpid', 'schoolRating']].copy()
        # Filter to only include considered properties
        schools = schools[schools['zpid'].isin(considered_properties)]
        school_ratings = schools.groupby('zpid')['schoolRating'].mean().reset_index()
        school_ratings.rename(columns={'schoolRating': 'avg_school_rating'}, inplace=True)
        df = pd.merge(df, school_ratings, on='zpid', how='left')

        # 9. Add nearby home comparison
        nearby = self.datasets['nearby_homes'][['zpid', 'priceComp']].copy()
        # Filter to only include considered properties
        nearby = nearby[nearby['zpid'].isin(considered_properties)]
        nearby_avg_price = nearby.groupby('zpid')['priceComp'].mean().reset_index()
        nearby_avg_price.rename(columns={'priceComp': 'nearby_avg_price'}, inplace=True)
        df = pd.merge(df, nearby_avg_price, on='zpid', how='left')
        df.loc[:, 'price_vs_nearby'] = df['price'] / df['nearby_avg_price']

        # 10. One-hot encoding
        for col in ['homeType', 'homeStatus', 'state']:
            if col in df.columns:
                top_values = df[col].value_counts().nlargest(5).index
                for val in top_values:
                    df.loc[:, f'{col}_{val}'] = (df[col] == val).astype('int8')

        # Drop original categorical columns to save memory
        df = df.drop(['homeType', 'homeStatus', 'state'], axis=1, errors='ignore')

        # 11. Handle missing values
        df.fillna({
            'price_yoy_change': 0,
            'valueIncreaseRate': df['valueIncreaseRate'].median(),
            'taxPaid': df['taxPaid'].median(),
            'rate': 0.05,  # Default mortgage rate
            'avg_school_rating': df['avg_school_rating'].median(),
            'nearby_avg_price': df['nearby_avg_price'].median(),
            'price_vs_nearby': 1,
            'pageViewCount': 0,
            'favoriteCount': 0
        }, inplace=True)

        # 12. Store property values by year for reward calculation
        for year in df['year'].unique():
            year_data = df[df['year'] == year][['zpid', 'price']]
            self.property_values_by_year[year] = dict(zip(year_data['zpid'].values, year_data['price'].values))

        # 13. Final cleanup
        df.replace([np.inf, -np.inf], np.finfo('float32').max, inplace=True)

        # Update main dataframe
        self.datasets['processed_data'] = df

        print(f"Feature engineering complete. Final dataframe shape: {df.shape}")
        print(f"Years available in processed data: {sorted(df['year'].unique())}")
        return df


    def prepare_state_features(self):
        """Prepare the state features for the DQN model"""
        if 'processed_data' not in self.datasets:
            print("Features not engineered. Call engineer_features() first.")
            return

        df = self.datasets['processed_data']

        # Check for and add default values for missing columns
        missing_columns = {
            'rate': 0.05,  # Default mortgage rate
            'pageViewCount': 0,  # Default page views
            'favoriteCount': 0   # Default favorites
        }

        for col, default_val in missing_columns.items():
            if col not in df.columns:
                print(f"Warning: Column '{col}' not found, adding with default value {default_val}")
                df[col] = default_val

        # Now proceed with feature selection
        feature_columns = [
            'price', 'livingArea', 'bedrooms', 'bathrooms', 'yearBuilt', 'age',
            'price_per_sqft', 'bed_bath_ratio', 'price_yoy_change', 'taxPaid',
            'valueIncreaseRate', 'rate', 'avg_school_rating', 'nearby_avg_price',
            'price_vs_nearby', 'pageViewCount', 'favoriteCount'
        ]
        # Add the one-hot encoded columns for homeType, homeStatus, and state
        one_hot_columns = [col for col in df.columns if col.startswith(('homeType_', 'homeStatus_', 'state_'))]
        feature_columns.extend(one_hot_columns)

        # Extract just the feature columns
        features_df = df[feature_columns + ['zpid', 'year']]

        # Scale the numerical features
        numerical_features = [
            'price', 'livingArea', 'bedrooms', 'bathrooms', 'age',
            'price_per_sqft', 'bed_bath_ratio', 'price_yoy_change', 'taxPaid',
            'valueIncreaseRate', 'rate', 'avg_school_rating', 'nearby_avg_price',
            'price_vs_nearby', 'pageViewCount', 'favoriteCount'
        ]

        # Fit scaler on training data only to prevent data leakage
        train_features = features_df[features_df['zpid'].isin(self.property_sets['train'])]
        self.scaler = StandardScaler()
        self.scaler.fit(train_features[numerical_features])

        # Transform all features
        features_df[numerical_features] = self.scaler.transform(features_df[numerical_features])

        # Store the prepared features
        self.datasets['state_features'] = features_df

        # Return the feature dimension for the DQN model
        self.state_size = len(feature_columns)
        print(f"State features prepared. State dimension: {self.state_size}")

        return features_df, self.state_size

    def get_properties_for_year(self, year, split):
        """Get properties available in a specific year for a specific data split"""
        if 'state_features' not in self.datasets:
            print("State features not prepared. Call prepare_state_features() first.")
            return None

        # Get properties for the given split
        split_properties = self.property_sets[split]

        # Get data for the given year filtered by split properties
        year_data = self.datasets['state_features'][
            (self.datasets['state_features']['year'] == year) &
            (self.datasets['state_features']['zpid'].isin(split_properties))
        ]

        return year_data

    def get_property_state(self, property_id, year):
        """Get the state representation for a specific property and year"""
        if 'state_features' not in self.datasets:
            print("State features not prepared. Call prepare_state_features() first.")
            return None

        # Get data for the specific property and year
        property_data = self.datasets['state_features'][
            (self.datasets['state_features']['zpid'] == property_id) &
            (self.datasets['state_features']['year'] == year)
        ]

        if property_data.empty:
            return None

        # Return state features (excluding zpid and year)
        state_features = property_data.drop(['zpid', 'year'], axis=1).values[0]

        return state_features

    def get_property_value(self, property_id, year):
        """Get the value of a property in a specific year"""
        return self.property_values_by_year.get(year, {}).get(property_id, None)


def reduce_memory_usage(df):
    """Reduces memory usage of a dataframe by downcasting numeric types."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage before optimization: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        # Skip categorical columns or convert them to ordered
        if isinstance(col_type, CategoricalDtype):
            # Option 1: Skip categorical columns entirely
            continue

            # Option 2: Make categorical columns ordered before operations
            # df[col] = df[col].cat.as_ordered()

        elif col_type != object:
            # Use safe min/max operations with explicit null handling
            if df[col].isna().any():  # Check for nulls first
                non_null_values = df[col].dropna()
                if len(non_null_values) > 0:  # Make sure we have values after dropping nulls
                    c_min = non_null_values.min()
                    c_max = non_null_values.max()
                else:
                    # If all values are null, use safe defaults
                    c_min = 0
                    c_max = 0
            else:
                # No nulls, proceed normally
                c_min = df[col].min()
                c_max = df[col].max()

            # Integer downcasting
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)

            # Float downcasting
            elif str(col_type)[:5] == 'float':
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)

        # For object columns that we want to convert to category
        elif df[col].nunique() < df.shape[0] * 0.5:
            # Handle null values in categorical data
            if df[col].isna().any():
                # Convert to object type first, fill nulls with placeholder, then convert to category
                df[col] = df[col].astype(object).fillna('Unknown').astype('category')
            else:
                # No nulls, safe to convert directly
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization: {end_mem:.2f} MB')
    print(f'Reduced by {100 * (start_mem - end_mem) / start_mem:.1f}%')

    return df

def train_agent(agent, data_processor, split='train', num_epochs=5):
    """Train the agent using the specified data split"""
    print(f"Training agent on {split} split for {num_epochs} epochs...")
    years = sorted(data_processor.datasets['price_history']['dateOfEvent'].dt.year.unique())
    
    # If no historical years, use current year from listings
    if len(years) == 0:
        years = [data_processor.datasets['property_listings']['year'].max()]
    
    # Force at least one training year
    years = years or [2024]  # Default to 2024 if no years found
    
    print(f"Training years: {years}")
    returns_history = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Reset agent's portfolio at the beginning of each epoch
        agent.portfolio = {}
        agent.cash = 1000000  # Reset starting cash
        agent.net_worth_history = []

        total_profit = 0

        # For each year in chronological order
        for i, year in enumerate(tqdm(years, desc=f"Processing years")):
            # Get available properties for this year and split
            year_data = data_processor.get_properties_for_year(year, split)

            if year_data is None or year_data.empty:
                print(f"No data available for year {year}, split {split}")
                continue

            # Track properties acted on this year to avoid duplicates
            acted_properties = set()

            # First, check if any properties in the portfolio should be sold
            properties_to_sell = []
            for prop_id, details in agent.portfolio.items():
                holding_years = year - details['purchase_year']

                # Check if we've held the property for the recommended period
                if holding_years >= details['recommended_holding_period']:
                    properties_to_sell.append(prop_id)

            # Sell properties that have reached their holding period
            for prop_id in properties_to_sell:
                current_value = data_processor.get_property_value(prop_id, year)

                if current_value is not None:
                    # Get property state for the selling decision
                    prop_state = data_processor.get_property_state(prop_id, year)

                    if prop_state is not None:
                        # Get agent's action (0: hold, 1: sell)
                        action = agent.act(prop_state)

                        if action == 1:  # Sell
                            profit = agent.update_portfolio(prop_id, 'sell', current_value, year)
                            total_profit += profit

                            # Calculate reward (profit as percentage of purchase price)
                            purchase_price = agent.portfolio.get(prop_id, {}).get('purchase_price', current_value)
                            reward = profit / purchase_price if purchase_price > 0 else 0

                            # Get the next state (could be None at the end of data)
                            next_state = prop_state  # Simplified; in real scenario would be updated state

                            # Determine if this is the end of the episode
                            done = (i == len(years) - 1)

                            # Agent learning step
                            agent.step(prop_state, np.array([action]), reward, next_state, done)

                            acted_properties.add(prop_id)

            # Then, consider buying new properties
            for _, row in year_data.iterrows():
                prop_id = row['zpid']

                # Skip if we already acted on this property this year
                if prop_id in acted_properties:
                    continue

                # Get property state
                prop_state = data_processor.get_property_state(prop_id, year)

                if prop_state is not None:
                    # Get agent's action (0: hold/don't buy, 1: buy)
                    action = agent.act(prop_state)

                    if action == 1:  # Buy
                        property_price = data_processor.get_property_value(prop_id, year)

                        if property_price is not None and agent.cash >= property_price:
                            success = agent.update_portfolio(prop_id, 'buy', property_price, year)

                            # Calculate immediate reward (small negative for buying to account for transaction costs)
                            reward = -0.01  # Small negative reward for buying

                            # Get the next state (could be None at the end of data)
                            next_state = prop_state  # Simplified

                            # Determine if this is the end of the episode
                            done = (i == len(years) - 1)

                            # Agent learning step
                            agent.step(prop_state, np.array([[action]]), reward, next_state, done)

                            acted_properties.add(prop_id)

            # Update agent's net worth at the end of each year
            current_property_values = {prop_id: data_processor.get_property_value(prop_id, year)
                                      for prop_id in agent.portfolio.keys()}
            current_property_values = {k: v for k, v in current_property_values.items() if v is not None}

            net_worth = agent.record_net_worth(current_property_values)
            print(f"Year {year} - Net Worth: ${net_worth:,.2f}, Cash: ${agent.cash:,.2f}, Properties: {len(agent.portfolio)}")

        if agent.net_worth_history:
            epoch_return = agent.net_worth_history[-1] - 1000000
        else:
            epoch_return = 0
            print("Warning: No net worth recorded this epoch")  # Final net worth minus initial cash
        returns_history.append(epoch_return)
        print(f"Epoch {epoch+1} Return: ${epoch_return:,.2f}")

        # Display detailed portfolio information after each epoch
        agent.display_portfolio_details(data_processor)

    return returns_history

def evaluate_agent(agent, data_processor, split='test'):
    """Evaluate the agent on the specified data split

    Args:
        agent (RealEstateAgent): The trained agent
        data_processor (RealEstateDataProcessor): The data processor
        split (str): The data split to use ('val', 'test', 'prod')
    """
    print(f"Evaluating agent on {split} split...")

    years = data_processor.years

    # Reset agent's portfolio for evaluation
    agent.portfolio = {}
    agent.cash = 1000000  # Reset starting cash
    agent.net_worth_history = []

    total_profit = 0
    transactions = []

    # For each year in chronological order
    for i, year in enumerate(tqdm(years, desc=f"Evaluating years")):
        # Get available properties for this year and split
        year_data = data_processor.get_properties_for_year(year, split)

        if year_data is None or year_data.empty:
            print(f"No data available for year {year}, split {split}")
            continue

        # Track properties acted on this year to avoid duplicates
        acted_properties = set()

        # First, check if any properties in the portfolio should be sold
        properties_to_sell = []
        for prop_id, details in agent.portfolio.items():
            holding_years = year - details['purchase_year']

            # Check if we've held the property for the recommended period
            if holding_years >= details['recommended_holding_period']:
                properties_to_sell.append(prop_id)

        # Sell properties that have reached their holding period
        for prop_id in properties_to_sell:
            current_value = data_processor.get_property_value(prop_id, year)

            if current_value is not None:
                # Get property state for the selling decision
                prop_state = data_processor.get_property_state(prop_id, year)

                if prop_state is not None:
                    # Get agent's action (0: hold, 1: sell) - in eval mode
                    action = agent.act(prop_state, eval_mode=True)

                    if action == 1:  # Sell
                        purchase_price = agent.portfolio[prop_id]['purchase_price']
                        purchase_year = agent.portfolio[prop_id]['purchase_year']
                        profit = agent.update_portfolio(prop_id, 'sell', current_value, year)
                        total_profit += profit

                        # Record the transaction
                        transactions.append({
                            'year': year,
                            'action': 'sell',
                            'property_id': prop_id,
                            'price': current_value,
                            'profit': profit,
                            'holding_period': year - purchase_year
                        })

                        acted_properties.add(prop_id)

        # Then, consider buying new properties
        for _, row in year_data.iterrows():
            prop_id = row['zpid']

            # Skip if we already acted on this property this year
            if prop_id in acted_properties:
                continue

            # Get property state
            prop_state = data_processor.get_property_state(prop_id, year)

            if prop_state is not None:
                # Get agent's action (0: hold/don't buy, 1: buy) - in eval mode
                action = agent.act(prop_state, eval_mode=True)

                if action == 1:  # Buy
                    property_price = data_processor.get_property_value(prop_id, year)

                    if property_price is not None and agent.cash >= property_price:
                        success = agent.update_portfolio(prop_id, 'buy', property_price, year)

                        if success:
                            # Record the transaction
                            transactions.append({
                                'year': year,
                                'action': 'buy',
                                'property_id': prop_id,
                                'price': property_price
                            })

                            acted_properties.add(prop_id)

        # Update agent's net worth at the end of each year
        current_property_values = {prop_id: data_processor.get_property_value(prop_id, year)
                                  for prop_id in agent.portfolio.keys()}
        current_property_values = {k: v for k, v in current_property_values.items() if v is not None}

        net_worth = agent.record_net_worth(current_property_values)
        print(f"Year {year} - Net Worth: ${net_worth:,.2f}, Cash: ${agent.cash:,.2f}, Properties: {len(agent.portfolio)}")

    # Calculate final return
    final_return = agent.net_worth_history[-1] - 1000000  # Final net worth minus initial cash
    roi = (final_return / 1000000) * 100  # ROI as percentage

    print(f"\nEvaluation Results on {split} split:")
    print(f"Final Net Worth: ${agent.net_worth_history[-1]:,.2f}")
    print(f"Total Return: ${final_return:,.2f}")
    print(f"ROI: {roi:.2f}%")
    print(f"Total Transactions: {len(transactions)}")

    # Convert transactions to DataFrame for analysis
    if transactions:
        transactions_df = pd.DataFrame(transactions)
        buy_count = len(transactions_df[transactions_df['action'] == 'buy'])
        sell_count = len(transactions_df[transactions_df['action'] == 'sell'])
        avg_profit = transactions_df[transactions_df['action'] == 'sell']['profit'].mean()
        avg_holding = transactions_df[transactions_df['action'] == 'sell']['holding_period'].mean()

        print(f"Buy Transactions: {buy_count}")
        print(f"Sell Transactions: {sell_count}")
        print(f"Average Profit per Sale: ${avg_profit:,.2f}")
        print(f"Average Holding Period: {avg_holding:.1f} years")

    # Plot net worth over time
    plt.figure(figsize=(12, 6))
    plt.plot(years[:len(agent.net_worth_history)], agent.net_worth_history)
    plt.title(f'Agent Net Worth Over Time ({split} split)')
    plt.xlabel('Year')
    plt.ylabel('Net Worth ($)')
    plt.grid(True)
    plt.savefig(f'agent_net_worth_{split}.png')
    plt.show()

    return {
        'final_net_worth': agent.net_worth_history[-1],
        'total_return': final_return,
        'roi': roi,
        'transactions': transactions
    }

def main():
    """Main function to orchestrate the complete workflow"""
    # Initialize data processor
    data_processor = RealEstateDataProcessor(data_dir='./')

    # Load and process data
    if not data_processor.load_datasets():
        print("Failed to load datasets. Exiting.")
        return

    # Split data by properties
    data_processor.split_data_by_properties()

    # Engineer features
    data_processor.engineer_features()
    print("TWAS SPLIT")
    # Prepare state features
    _, state_size = data_processor.prepare_state_features()
    print("TWAS PREPARED")
    # Define action space: 0 = Hold/Don't Buy, 1 = Buy/Sell
    action_size = 2

    # Initialize the agent
    agent = RealEstateAgent(state_size=state_size, action_size=action_size)
    print("HEEEEEEEEEEEERE")
    # Train the agent
    train_returns = train_agent(agent, data_processor, split='train', num_epochs=5)

    # Evaluate on validation set
    val_results = evaluate_agent(agent, data_processor, split='val')

    # If validation results are satisfactory, evaluate on test set
    test_results = evaluate_agent(agent, data_processor, split='test')

    # Finally, evaluate on production set
    prod_results = evaluate_agent(agent, data_processor, split='prod')

    # Compare results across splits
    results = {
        'val': val_results,
        'test': test_results,
        'prod': prod_results
    }

    # Print comparison
    print("\nPerformance Comparison:")
    print("Split\tROI\tFinal Net Worth")
    for split, res in results.items():
        print(f"{split}\t{res['roi']:.2f}%\t${res['final_net_worth']:,.2f}")

    print("\nTraining complete!")

if __name__ == "__main__":
    main()
